In [85]:
import pandas as pd

In [86]:
# TODO: create seperate a file for preprocessed data??
df = pd.read_csv('data/average-monthly-surface-temperature.csv')

In [87]:
df.sample(5)

,Entity,Code,year,Day,Average surface temperature,Average surface temperature.1
115404,Mauritania,MRT,1952,1952-01-15,15.306032,25.682121
120680,Montenegro,MNE,1966,1966-09-15,14.592620,8.542099
102931,Kyrgyzstan,KGZ,2017,2017-08-15,11.369824,0.497010
70615,Georgia,GEO,1959,1959-08-15,17.007864,6.432468
74547,Greenland,GRL,1947,1947-04-15,-24.363405,-18.887688


In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198900 entries, 0 to 198899
Data columns (total 6 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Entity                         198900 non-null  object 
 1   Code                           198900 non-null  object 
 2   year                           198900 non-null  int64  
 3   Day                            198900 non-null  object 
 4   Average surface temperature    198900 non-null  float64
 5   Average surface temperature.1  198900 non-null  float64
dtypes: float64(2), int64(1), object(3)
memory usage: 9.1+ MB


# Missing values

In [89]:
missing_values = df.isnull().sum()
print("Missing Values:")
print(missing_values)

# there are no missing values

Missing Values:
Entity                           0
Code                             0
year                             0
Day                              0
Average surface temperature      0
Average surface temperature.1    0
dtype: int64


# Duplicates

In [90]:
duplicate_rows = df[df.duplicated()]
print("Duplicate Rows:")
print(duplicate_rows)

# there are no duplicate rows

Duplicate Rows:
Empty DataFrame
Columns: [Entity, Code, year, Day, Average surface temperature, Average surface temperature.1]
Index: []


# Convert Data Types

In [91]:
df["Day"] = pd.to_datetime(df["Day"])
df

,Entity,Code,year,Day,Average surface temperature,Average surface temperature.1
0,Afghanistan,AFG,1940,1940-01-15,-2.032494,11.327695
1,Afghanistan,AFG,1940,1940-02-15,-0.733503,11.327695
2,Afghanistan,AFG,1940,1940-03-15,1.999134,11.327695
3,Afghanistan,AFG,1940,1940-04-15,10.199754,11.327695
4,Afghanistan,AFG,1940,1940-05-15,17.942135,11.327695
...,...,...,...,...,...,...
198895,Zimbabwe,ZWE,2024,2024-08-15,20.559408,22.921250
198896,Zimbabwe,ZWE,2024,2024-09-15,23.642931,22.921250
198897,Zimbabwe,ZWE,2024,2024-10-15,24.407030,22.921250
198898,Zimbabwe,ZWE,2024,2024-11-15,25.672321,22.921250


# Other

In [92]:
df["Code"].unique()

array(['AFG', 'ALB', 'DZA', 'ASM', 'AND', 'AGO', 'AIA', 'ATG', 'ARG',
       'ARM', 'AUS', 'AUT', 'AZE', 'BHS', 'BHR', 'BGD', 'BLR', 'BEL',
       'BLZ', 'BEN', 'BTN', 'BOL', 'BIH', 'BWA', 'BRA', 'BRN', 'BGR',
       'BFA', 'BDI', 'KHM', 'CMR', 'CAN', 'CPV', 'CYM', 'CAF', 'TCD',
       'CHL', 'CHN', 'COL', 'COM', 'COG', 'COK', 'CRI', 'CIV', 'HRV',
       'CUB', 'CYP', 'CZE', 'COD', 'DNK', 'DJI', 'DOM', 'TLS', 'ECU',
       'EGY', 'SLV', 'GNQ', 'ERI', 'EST', 'SWZ', 'ETH', 'FLK', 'FRO',
       'FJI', 'FIN', 'FRA', 'PYF', 'GAB', 'GMB', 'GEO', 'DEU', 'GHA',
       'GRC', 'GRL', 'GTM', 'GIN', 'GNB', 'GUY', 'HTI', 'HMD', 'HND',
       'HKG', 'HUN', 'ISL', 'IND', 'IDN', 'IRN', 'IRQ', 'IRL', 'IMN',
       'ISR', 'ITA', 'JAM', 'JPN', 'JOR', 'KAZ', 'KEN', 'KIR', 'OWID_KOS',
       'KWT', 'KGZ', 'LAO', 'LVA', 'LBN', 'LSO', 'LBR', 'LBY', 'LTU',
       'LUX', 'MDG', 'MWI', 'MYS', 'MLI', 'MRT', 'MUS', 'MEX', 'MDA',
       'MNG', 'MNE', 'MAR', 'MOZ', 'MMR', 'NAM', 'NPL', 'NLD', 'NCL',
       'NZL', '

In [93]:
country_code = "MLI"
df.loc[df["Code"] == country_code]

,Entity,Code,year,Day,Average surface temperature,Average surface temperature.1
114240,Mali,MLI,1940,1940-01-15,19.892700,27.311247
114241,Mali,MLI,1940,1940-02-15,22.298273,27.311247
114242,Mali,MLI,1940,1940-03-15,25.804117,27.311247
114243,Mali,MLI,1940,1940-04-15,29.059055,27.311247
114244,Mali,MLI,1940,1940-05-15,32.057050,27.311247
...,...,...,...,...,...,...
115255,Mali,MLI,2024,2024-08-15,31.823093,29.663470
115256,Mali,MLI,2024,2024-09-15,30.517645,29.663470
115257,Mali,MLI,2024,2024-10-15,30.693073,29.663470
115258,Mali,MLI,2024,2024-11-15,26.287638,29.663470
